In [3]:
using JuMP, Gurobi

INFO: Precompiling module Gurobi.


In [ ]:


# Task/Worker Parameters
###############################

## We require 5 of task_1, 1 of task_2, etc.
N = [50,40,100]  #tasks
n = maximum(N) #max number of tasks for a category
v = size(N,1) # number of types of tasks

## Each row of W is a worker, and the column is the time required for the worker to complete that task
#W = hcat(ones(w), 3*ones(w), 2*ones(w)) # 1 worker who completes task 1 in 1 time, tasks 2 in three time, etc.
W = [1 3 4;
     5 6 1;]
w = size(W,1) #number of workers


T = convert(Int64,maximum(W)*sum(N)) #upper bound on project completion

###############################



m = Model(solver=GurobiSolver())

@variable(m, x[1:n, 1:v, 1:w, 1:T], Bin) # true when a worker starts a task
@variable(m, z[1:n, 1:v, 1:w, 1:T], Bin) #true if a worker is working on a task (extends x with task length)

@variable(m, last)

#If a worker starts a task at time t, z if true until the tasks completes
for i=1:n
    for j=1:v
        for k=1:w
            for t=1:T
                #@constraint(m, [i=1:n,j=1:v,k=1:w, t=1:T], z[i,j,k,t+l] == x[i,j,k,t] for l=1:W[k,j])
                if t < T-convert(Int64,W[k,j])
                    @constraint(m, [l=1:convert(Int64,W[k,j])], z[i,j,k,t+l-1] >= x[i,j,k,t])  
                else
                    @constraint(m, [l=1:T-t], z[i,j,k,t+l-1] >= x[i,j,k,t])  
                end
            end
        end
    end
end

#only one task in z can be worked simultaneously
@constraint(m, [k=1:w, t=1:T], sum(z[:,:,k,t]) <= 1)

#expression is used as an epigraph to find the end time of the last task
@expression(m, end_t[j=1:n, k=1:v, i=1:w], sum((x[j,k,i,t])*(t+W[i,k]) for t=1:T))
@expression(m, start_t[j=1:n, k=1:v, i=1:w], sum((x[j,k,i,t])*(t) for t=1:T))

# at most one job per timestep per person
@constraint(m, [t=1:T, i=1:w], sum(x[:, :, i, t]) <= 1)

# each task is only worked on once and all tasks are completed
@constraint(m, [j=1:n, i=1:v], sum(x[j, i, :, :]) <= 1)
@constraint(m, [i=1:v], sum(x[:,i,:,:]) == N[i])

# min { max_j end_t[j] }
@constraint(m, [j=1:n, k=1:v, i=1:w], last >= end_t[j,k,i])

@objective(m, Min, last)


@time(solve(m))
getobjectivevalue(m)

In [26]:
#Gives the times during which workers are working on tasks
getvalue(z)

10×3×2×114 Array{Float64,4}:
[:, :, 1, 1] =
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

[:, :, 2, 1] =
 -0.0  0.0  -0.0
 -0.0  0.0  -0.0
 -0.0  0.0   1.0
 -0.0  0.0  -0.0
 -0.0  0.0  -0.0
 -0.0  0.0  -0.0
 -0.0  0.0  -0.0
 -0.0  0.0  -0.0
 -0.0  0.0  -0.0
 -0.0  0.0  -0.0

[:, :, 1, 2] =
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

[:, :, 2, 2] =
 -0.0  0.0  1.0
 -0.0  0.0  0.0
 -0.0  0.0  0.0
 -0.0  0.0  0.0
 -0.0  0.0  0.0
 -0.0  0.0  0.0
 -0.0  0.0  0.0
 -0.0  0.0  0.0
 -0.0  0.0  0.0
 -0.0  0.0  0.0

[:, :, 1, 3] =
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

[:, :, 2, 3] =
 -0.0  0.0  0.0
 -0.0  0.0  0.0
 -0.0  0.0  0.0
 -0.0  0.0  0.0
 -0.0  0.0  0.0
 -0.0  0.0  0.0


In [15]:
#Gives the times a worker starts/finishes each task

#getvalue(start_t)
getvalue(end_t)

5×3×2 Array{Float64,3}:
[:, :, 1] =
 0.0  0.0  0.0
 7.0  0.0  0.0
 3.0  0.0  0.0
 8.0  6.0  0.0
 2.0  0.0  0.0

[:, :, 2] =
 8.0  0.0  0.0
 0.0  0.0  2.0
 0.0  0.0  9.0
 0.0  0.0  3.0
 0.0  0.0  0.0